In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from keras.models import Model, load_model, save_model
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add,merge,Flatten,ZeroPadding2D,GlobalAveragePooling2D\
,Dense,ZeroPadding2D
from keras.engine import get_source_inputs
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers.pooling import MaxPooling2D,GlobalAveragePooling2D,AveragePooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras import optimizers
from keras.applications.imagenet_utils import _obtain_input_shape
import keras
from keras.utils import get_file

from keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Model
from keras.optimizers import SGD, adam

from skimage.transform import resize
import tensorflow as tf
import gc

Using TensorFlow backend.


In [3]:
machine_labels = pd.read_csv("train_machine_labels.csv", engine='python')
human_labels = pd.read_csv("train_human_labels.csv", engine='python')
classes_trainable = pd.read_csv("classes-trainable.csv", engine='python')
classes_all = pd.read_csv("class-descriptions.csv", engine='python')
training_sets = pd.read_csv("train-images-boxable-with-rotation.csv", engine='python')

In [4]:
classes_trainable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7178 entries, 0 to 7177
Data columns (total 1 columns):
label_code    7178 non-null object
dtypes: object(1)
memory usage: 56.2+ KB


In [10]:
training_sets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1743042 entries, 0 to 1743041
Data columns (total 12 columns):
ImageID               object
Subset                object
OriginalURL           object
OriginalLandingURL    object
License               object
AuthorProfileURL      object
Author                object
Title                 object
OriginalSize          int64
OriginalMD5           object
Thumbnail300KURL      object
Rotation              float64
dtypes: float64(1), int64(1), object(10)
memory usage: 159.6+ MB


In [45]:
machine_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15259186 entries, 0 to 15259185
Data columns (total 4 columns):
ImageID       object
Source        object
LabelName     object
Confidence    float64
dtypes: float64(1), object(3)
memory usage: 465.7+ MB


In [60]:
np.unique(machine_labels.ImageID).shape

(1708875,)

In [62]:
np.unique(human_labels.ImageID).shape

(1743042,)

In [46]:
human_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8036466 entries, 0 to 8036465
Data columns (total 4 columns):
ImageID       object
Source        object
LabelName     object
Confidence    int64
dtypes: int64(1), object(3)
memory usage: 245.3+ MB


In [65]:
classes_trainable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7178 entries, 0 to 7177
Data columns (total 1 columns):
label_code    7178 non-null object
dtypes: object(1)
memory usage: 56.2+ KB


In [74]:
human_labels.LabelName

0           /m/01kcnl
1            /m/012mj
2           /m/012yh1
3           /m/014sv8
4           /m/016c68
5           /m/016q19
6           /m/019nj4
7           /m/019_nn
8           /m/019sc6
9           /m/01bl7v
10          /m/01bsxb
11         /m/01c5nlx
12          /m/01d40f
13          /m/01d74z
14          /m/01fklc
15          /m/01g317
16          /m/01mzpv
17           /m/01tv9
18           /m/01vkl
19           /m/01ykh
20          /m/021dfr
21         /m/026pc7x
22           /m/0270h
23           /m/0271t
24         /m/0283dt1
25           /m/0284d
26           /m/02cwm
27          /m/02czv8
28          /m/02f6b9
29          /m/02jwqh
              ...    
8036436     /m/01g317
8036437    /m/03bt1vf
8036438     /m/0463sg
8036439      /m/09j2d
8036440     /m/0c9ph5
8036441      /m/0dzct
8036442     /m/01g317
8036443    /m/03bt1vf
8036444      /m/04yx4
8036445     /m/01g317
8036446     /m/01prls
8036447      /m/04_sv
8036448      /m/04yx4
8036449      /m/07yv9
8036450   

In [7]:
human_labels.columns

Index(['ImageID', 'Source', 'LabelName', 'Confidence'], dtype='object')

In [48]:
uni_confi_machine = np.unique(machine_labels.Confidence)

In [50]:
machine_confident_labels = []
machine_confident_ids = []
for i,j in enumerate(machine_labels.loc[:, 'Confidence']):
    if j == 1:
        machine_confident_ids.append(machine_labels.loc[i, "ImageID"])
        machine_confident_labels.append(machine_labels.loc[i, "LabelName"])

In [55]:
uni_machine_confident_labels = np.unique(machine_confident_labels)
print(uni_machine_confident_labels)

['/m/010jjr' '/m/010l12' '/m/011_lb' ... '/m/0x44' '/m/0ytgt' '/m/0zg7256']


In [57]:
uni_machine_confident_ids = np.unique(machine_confident_ids)
print(uni_machine_confident_ids)

['000020780ccee28d' '000023aa04ab09ed' '0000253ea4ecbf19' ...
 'ffffb23e3d693832' 'ffffd00a2aa77064' 'ffffebddbb8bba21']


In [58]:
uni_machine_confident_ids.shape

(435436,)

In [59]:
uni_machine_confident_labels.shape

(1168,)

In [52]:
machine_confident_ids

['000020780ccee28d',
 '000023aa04ab09ed',
 '0000253ea4ecbf19',
 '0000271195f2c007',
 '0000286a5c6a3eb5',
 '0000313e5dccf13b',
 '0000333f08ced1cd',
 '0000333f08ced1cd',
 '0000375a83c19042',
 '0000375a83c19042',
 '0000418f7ade0445',
 '0000432bf1241812',
 '000046611bd6b353',
 '000046611bd6b353',
 '000046611bd6b353',
 '000046611bd6b353',
 '00004668fd1ee4e7',
 '0000575f5a03db70',
 '0000575f5a03db70',
 '0000575f5a03db70',
 '0000587b9647679e',
 '00005bf623ff1ac2',
 '00005bf623ff1ac2',
 '00005bf623ff1ac2',
 '00005bf623ff1ac2',
 '0000626c3e317247',
 '00006c07d2b033d1',
 '00006f304dd2d92d',
 '000077165b72a53b',
 '000077165b72a53b',
 '00007f25a6bac8ed',
 '000094eaec4680b3',
 '000096726fd6c6c8',
 '000096726fd6c6c8',
 '000096726fd6c6c8',
 '0000a1b2fba255e9',
 '0000a4e648c5897f',
 '0000b9fcba019d36',
 '0000b9fcba019d36',
 '0000bd371f4b1bba',
 '0000c035a08c3770',
 '0000c035a08c3770',
 '0000cf997957ffc1',
 '0000cf997957ffc1',
 '0000d01325742829',
 '0000d01325742829',
 '0000d59fa570d973',
 '0000dc465ec

In [43]:
uni_human = np.unique(human_labels.LabelName)
uni_machine = np.unique(machine_labels.LabelName)

In [32]:
samples = pd.read_csv("stage_1_sample_submission.csv")

In [40]:
samples["labels"][0].split(" ")

['/m/0sgh53y', '/m/0g4cd0']

In [17]:
uni_confidence = np.unique(machine_labels.Confidence)

In [18]:
uni_confidence

array([0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

# data split

In [42]:
uni_human.shape

(18155,)

In [44]:
uni_machine.shape

(7509,)

In [71]:
trainable_not_in_human = []
for i in classes_trainable.label_code:
    if i not in uni_human:
        trainable_not_in_human.append(i)

In [72]:
trainable_not_in_human

['/m/01dgzv', '/m/01wr8', '/m/04q347y', '/m/06f8q', '/m/07r2x', '/m/0brl6']

In [9]:
labels = {}
lst = list(classes_trainable.label_code)
for i,j in enumerate(human_labels.loc[:, "LabelName"]):
    if j in lst:
        id_ = human_labels.loc[i, "ImageID"]
        if id_ in labels:
            labels[id_] += " "
            labels[id_] += j
        else:
            labels[id_] = j

In [19]:
trainable_not_in_human = ['/m/01dgzv', '/m/01wr8', '/m/04q347y', '/m/06f8q', '/m/07r2x', '/m/0brl6']
for i,j in enumerate(machine_labels.loc[:, "LabelName"]):
    if j in trainable_not_in_human:
        id_ = machine_labels.loc[i, "ImageID"]
        confidence = machine_labels.loc[i, "Confidence"]
        if confidence > 0.7:
            if id_ in labels_machine:
                labels[id_] += " "
                labels[id_] += j
            else:
                labels[id_] = j

In [21]:
len(list(labels.keys()))

1741975

In [25]:
labels_freq = {}

lst_1 = list(classes_trainable.label_code)
for i in lst_1:
    labels_freq[i] = 0
    
for i in labels:
    lst = labels[i].split()
    for j in lst:
        labels_freq[j] += 1
        

In [37]:
array_freq = np.array(list(labels_freq.values()))
lst_freq= list(labels_freq.values())

In [60]:
lst_1 = list(classes_trainable.label_code)
labels_selected = {}
labels_times = {}
for i in lst_1:
    labels_times[i] = 0
    
for j in labels:
    labels_lst = labels[j].split()
    flag = False
    for i in labels_lst:
        if labels_times[i] < 15:
            flag = True
        labels_times[i] += 1
    if flag == True:
        labels_selected[j] = labels[j]

In [61]:
len(list(labels_selected.keys()))

50425

In [55]:
lst_1 = list(classes_trainable.label_code)
labels_valid = {}
labels_times = {}
for i in lst_1:
    labels_times[i] = 0
    
for j in labels:
    labels_lst = labels[j].split()
    flag = False
    for i in labels_lst:
        if labels_times[i] < 1:
            flag = True
        labels_times[i] += 1
    if flag == True:
        labels_valid[j] = labels[j]

In [56]:
len(list(labels_valid.keys()))

3907

In [62]:
training = pd.DataFrame.from_dict(labels_selected,orient='index')
training.index.names = ['ImageID']
training.columns = ['labels']
training.to_csv("training.csv")

In [59]:
valid = pd.DataFrame.from_dict(labels_valid,orient='index')
valid.index.names = ['ImageID']
valid.columns = ['labels']
valid.to_csv("valid.csv")